# DATA

In [1]:
database = '/content/copa_cai_aimar_sporting_club.db'

# PARTIDOS 2023

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(database)
query = "SELECT * FROM partidos_2023"
df = pd.read_sql_query(query, conn, index_col='id_partido')
conn.close()

df.head(5)

,fecha,hora,fecha_torneo,zona,etapa,equipo_local,equipo_visitante,resultado,goles_local,goles_visitante,amonestados_local,amonestados_visitante,expulsados_local,expulsados_visitante,penales,penales_local,penales_visitante,pasa_ronda
id_partido,,,,,,,,,,,,,,,,,,
1,05-12-2023,20:00,1,a,regular,RENTISTAS,UNION SF,visitante,2,3,1,1,0,0,None,NaN,NaN,None
2,05-12-2023,21:00,1,a,regular,COMUNICACIONES,RIVER PLATE,visitante,0,2,1,1,0,0,None,NaN,NaN,None
3,05-12-2023,22:00,1,a,regular,A.D.I.U.R.,COLON,visitante,0,3,0,3,0,0,None,NaN,NaN,None
4,05-12-2023,20:00,1,b,regular,PAYSANDU,O'HIGGINS,visitante,0,2,1,0,0,0,None,NaN,NaN,None
5,05-12-2023,21:00,1,b,regular,TALLERES,COLO COLO,visitante,1,2,0,1,0,0,None,NaN,NaN,None


# FUNCION PARA GENERAR LA TABLA DE POSICIONES

In [3]:
import pandas as pd

def generar_tabla_posiciones(df):
    # Crear una lista con los equipos únicos
    equipos = pd.concat([df['equipo_local'], df['equipo_visitante']]).unique()

    # Crear un DataFrame para la tabla de posiciones
    tabla_posiciones = pd.DataFrame({'equipo': equipos})

    # Inicializar las columnas de la tabla de posiciones
    tabla_posiciones['partidos_jugados'] = 0
    tabla_posiciones['ganados'] = 0
    tabla_posiciones['empatados'] = 0
    tabla_posiciones['perdidos'] = 0
    tabla_posiciones['goles_a_favor'] = 0
    tabla_posiciones['goles_en_contra'] = 0
    tabla_posiciones['diferencia_goles'] = 0
    tabla_posiciones['puntos'] = 0
    tabla_posiciones['tarjetas_amarillas'] = 0
    tabla_posiciones['expulsiones'] = 0

    # Actualizar las estadísticas de cada equipo en la tabla de posiciones
    for index, partido in df.iterrows():
        local = partido['equipo_local']
        visitante = partido['equipo_visitante']
        resultado = partido['resultado']
        goles_local = partido['goles_local']
        goles_visitante = partido['goles_visitante']
        tarjetas_local = partido['amonestados_local']
        tarjetas_visitante = partido['amonestados_visitante']
        expulsiones_local = partido['expulsados_local']
        expulsiones_visitante = partido['expulsados_visitante']

        # Verificar si hay datos en la columna 'resultado' y si son 'local', 'visitante' o 'empate'
        if pd.notna(resultado) and resultado in ['local', 'visitante', 'empate']:
            # Actualizar partidos jugados
            tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'partidos_jugados'] += 1
            tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'partidos_jugados'] += 1

            # Actualizar goles a favor y en contra
            tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'goles_a_favor'] += goles_local
            tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'goles_a_favor'] += goles_visitante
            tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'goles_en_contra'] += goles_visitante
            tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'goles_en_contra'] += goles_local

            # Actualizar partidos ganados, empatados y perdidos
            if resultado == 'local':
                tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'ganados'] += 1
                tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'perdidos'] += 1
            elif resultado == 'visitante':
                tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'perdidos'] += 1
                tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'ganados'] += 1
            else:
                tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'empatados'] += 1
                tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'empatados'] += 1

            # Actualizar tarjetas amarillas y expulsiones
            tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'tarjetas_amarillas'] += tarjetas_local
            tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'tarjetas_amarillas'] += tarjetas_visitante
            tabla_posiciones.loc[tabla_posiciones['equipo'] == local, 'expulsiones'] += expulsiones_local
            tabla_posiciones.loc[tabla_posiciones['equipo'] == visitante, 'expulsiones'] += expulsiones_visitante

    # Calcular la diferencia de goles y puntos
    tabla_posiciones['diferencia_goles'] = tabla_posiciones['goles_a_favor'] - tabla_posiciones['goles_en_contra']
    tabla_posiciones['puntos'] = tabla_posiciones['ganados'] * 3 + tabla_posiciones['empatados']

    # Ordenar la tabla de posiciones
    tabla_posiciones = tabla_posiciones.sort_values(by=['puntos', 'diferencia_goles', 'goles_a_favor'], ascending=False).reset_index(drop=True)

    return tabla_posiciones


## TABLA DE POSCIONES GENERAL

In [4]:
tabla_posiciones_general = generar_tabla_posiciones(df)
tabla_posiciones_general.insert(0, 'posicion', range(1, 1 + len(tabla_posiciones_general)))
tabla_posiciones_general.head(10)

,posicion,equipo,partidos_jugados,ganados,empatados,perdidos,goles_a_favor,goles_en_contra,diferencia_goles,puntos,tarjetas_amarillas,expulsiones
0,1,COLO COLO,7,6,1,0,17,3,14,19,6,0
1,2,RIVER PLATE,7,6,0,1,16,2,14,18,3,0
2,3,NEWELL'S,7,4,2,1,10,5,5,14,5,0
3,4,O'HIGGINS,7,4,1,2,9,3,6,13,1,0
4,5,TALLERES,7,3,2,2,13,5,8,11,2,0
5,6,COLON,7,3,1,3,8,5,3,10,14,1
6,7,A.D.I.U.R.,7,3,1,3,5,9,-4,10,1,1
7,8,COMUNICACIONES,7,2,1,4,2,6,-4,7,3,0
8,9,UNION SF,7,2,1,4,6,14,-8,7,4,0
9,10,SPORTING,7,1,2,4,5,20,-15,5,6,1


# FUNCION PARA CALCULAR Y AGREGAR PROMEDIOS




In [5]:
def calcular_promedios(tabla_posiciones):
    # Calcular promedios
    promedio_ganados = tabla_posiciones['ganados'].mean()
    promedio_empatados = tabla_posiciones['empatados'].mean()
    promedio_perdidos = tabla_posiciones['perdidos'].mean()
    promedio_goles_a_favor = tabla_posiciones['goles_a_favor'].mean()
    promedio_goles_en_contra = tabla_posiciones['goles_en_contra'].mean()
    promedio_diferencia_goles = tabla_posiciones['diferencia_goles'].mean()
    promedio_puntos = tabla_posiciones['puntos'].mean()
    promedio_tarjetas_amarillas= tabla_posiciones['tarjetas_amarillas'].mean()
    promedio_expulsiones = tabla_posiciones['expulsiones'].mean()

    # Crear nueva fila con los promedios
    fila_promedios = {
        'equipo': 'PROMEDIO',
        'ganados': promedio_ganados,
        'empatados': promedio_empatados,
        'perdidos': promedio_perdidos,
        'goles_a_favor': promedio_goles_a_favor,
        'goles_en_contra': promedio_goles_en_contra,
        'diferencia_goles': promedio_diferencia_goles,
        'puntos': promedio_puntos,
        'tarjetas_amarillas': promedio_tarjetas_amarillas,
        'expulsiones': promedio_expulsiones
    }

    # Agregar la fila de promedios al DataFrame
    tabla_con_promedios = tabla_posiciones.append(fila_promedios, ignore_index=True)

    # Convertir las columnas relevantes a tipo entero
    columnas_enteras = ['ganados', 'empatados', 'perdidos', 'goles_a_favor', 'goles_en_contra', 'diferencia_goles', 'puntos', 'tarjetas_amarillas', 'expulsiones']
    tabla_con_promedios[columnas_enteras] = tabla_con_promedios[columnas_enteras].astype(int)

    return tabla_con_promedios


## TABLA DE POSCIONES CON PROMEDIOS

In [6]:
tabla_posciones_general_con_promedios = calcular_promedios(tabla_posiciones_general)
tabla_posciones_general_con_promedios

<ipython-input-5-bc503b2db75c>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla_con_promedios = tabla_posiciones.append(fila_promedios, ignore_index=True)


,posicion,equipo,partidos_jugados,ganados,empatados,perdidos,goles_a_favor,goles_en_contra,diferencia_goles,puntos,tarjetas_amarillas,expulsiones
0,1.0,COLO COLO,7.0,6,1,0,17,3,14,19,6,0
1,2.0,RIVER PLATE,7.0,6,0,1,16,2,14,18,3,0
2,3.0,NEWELL'S,7.0,4,2,1,10,5,5,14,5,0
3,4.0,O'HIGGINS,7.0,4,1,2,9,3,6,13,1,0
4,5.0,TALLERES,7.0,3,2,2,13,5,8,11,2,0
5,6.0,COLON,7.0,3,1,3,8,5,3,10,14,1
6,7.0,A.D.I.U.R.,7.0,3,1,3,5,9,-4,10,1,1
7,8.0,COMUNICACIONES,7.0,2,1,4,2,6,-4,7,3,0
8,9.0,UNION SF,7.0,2,1,4,6,14,-8,7,4,0
9,10.0,SPORTING,7.0,1,2,4,5,20,-15,5,6,1


## TABLA DE POSCIONES GENERAL EN ETAPA REGULAR

In [7]:
df_etapa_regular = df[df['etapa'] == 'regular']
tabla_posciones_general_con_promedios_etapa_regular = generar_tabla_posiciones(df_etapa_regular)
tabla_posciones_general_con_promedios_etapa_regular.insert(0, 'posicion', range(1, 1 + len(tabla_posciones_general_con_promedios_etapa_regular)))
# funcion para calcular promedios
tabla_posciones_general_con_promedios_etapa_regular = calcular_promedios(tabla_posciones_general_con_promedios_etapa_regular)
tabla_posciones_general_con_promedios_etapa_regular

<ipython-input-5-bc503b2db75c>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla_con_promedios = tabla_posiciones.append(fila_promedios, ignore_index=True)


,posicion,equipo,partidos_jugados,ganados,empatados,perdidos,goles_a_favor,goles_en_contra,diferencia_goles,puntos,tarjetas_amarillas,expulsiones
0,1.0,RIVER PLATE,5.0,5,0,0,14,0,14,15,2,0
1,2.0,COLO COLO,5.0,5,0,0,15,2,13,15,3,0
2,3.0,COLON,5.0,3,1,1,7,2,5,10,12,0
3,4.0,NEWELL'S,5.0,3,1,1,8,4,4,10,3,0
4,5.0,O'HIGGINS,5.0,3,0,2,7,2,5,9,0,0
5,6.0,TALLERES,5.0,2,1,2,10,5,5,7,0,0
6,7.0,COMUNICACIONES,5.0,2,0,3,2,5,-3,6,2,0
7,8.0,UNION SF,5.0,2,0,3,5,10,-5,6,2,0
8,9.0,A.D.I.U.R.,5.0,1,1,3,2,9,-7,4,1,1
9,10.0,SPORTING,5.0,1,0,4,4,19,-15,3,4,0


## TABLA PARA ZONA A Y B

In [9]:
# zona A
tabla_zona_a = df[df['zona'] == 'a']
tabla_posiciones_a = generar_tabla_posiciones(tabla_zona_a)
tabla_posiciones_a.insert(0, 'posicion', range(1, 1 + len(tabla_posiciones_a)))

# zona B
tabla_zona_b = df[df['zona'] == 'b']
tabla_posiciones_b = generar_tabla_posiciones(tabla_zona_b)
tabla_posiciones_b.insert(0, 'posicion', range(1, 1 + len(tabla_posiciones_b)))

tabla_posiciones_a
tabla_posiciones_b

,posicion,equipo,partidos_jugados,ganados,empatados,perdidos,goles_a_favor,goles_en_contra,diferencia_goles,puntos,tarjetas_amarillas,expulsiones
0,1,COLO COLO,5,5,0,0,15,2,13,15,3,0
1,2,NEWELL'S,5,3,1,1,8,4,4,10,3,0
2,3,O'HIGGINS,5,3,0,2,7,2,5,9,0,0
3,4,TALLERES,5,2,1,2,10,5,5,7,0,0
4,5,SPORTING,5,1,0,4,4,19,-15,3,4,0
5,6,PAYSANDU,5,0,0,5,1,13,-12,0,3,0


# DESCARGAS



In [10]:
# tabla pura partidos_2023
df.to_csv('partidos_2023.csv', index=False)

# tabla de posciones general
tabla_posiciones_general.to_csv('tabla_posiciones_general.csv', index=False)

# tabla de posciones general con promedios
tabla_posciones_general_con_promedios.to_csv('tabla_posciones_general_con_promedios.csv', index=False)

# tabla de posciones general con promedios y etapa regular
tabla_posciones_general_con_promedios_etapa_regular.to_csv('tabla_posciones_general_con_promedios_etapa_regular.csv', index=False)

# tabla de posciones zona A y B
tabla_posiciones_a.to_csv('tabla_posiciones_zona_a.csv')
tabla_posiciones_b.to_csv('tabla_posiciones_zona_b.csv')